# Multivariate association with MASH
This note book document the process of meta analysis using mash. 

The output of association scan, and optionally the output from METAL, will undergo a reformmating process, and then fed into the MASH module to generate
1. MASH result in vcf format
2. Mixture prior for MvSuSIE_RSS

# Data Processing

## Allele flip QC

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/misc/yml_generator.ipynb yml_list \
--sumstat_list_path /mnt/mfs/statgen/xqtl_workflow_testing/success_example/testing_no_cov/QTL_association/sumstat_list  \
--wd ./  &

nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/misc/allele_flip_qc.ipynb \
--yml_list  /mnt/mfs/statgen/xqtl_workflow_testing/module/Merge/yml_list.txt  \
--cwd ./  &

## Merging and partitioning
This step takes in the qced sumstat from the previouse step and generate a merged rds for each gene. To feed into downstream analysis 

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/multivariate/MASH/sumstat_merger.ipynb merge \
--yml_list    \
--wd ./  &

## Extract effect


In [ ]:
[extract_effects]
input: output_from("merge_and_alleleQC")["merged_analysis_unit"]
output: extracted_effect = f'{wd}/sumstat/{Theme_prefix}.rds'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/Reformatting/Signal_Extraction.ipynb extract_effects \
            --cwd $[wd]/sumstat/ \
            --container $[container] \
            --name $[Theme_prefix] \
            --analysis_units $[_input] \
            -J $[J] -q $[queue] -c $[yml]

# Mash Analysis

## Factor analysis and MASH Model

In [ ]:
[MASH_FLASH]
parameter: effect_model = 'EZ'
parameter: mixture_components = ['flash', 'flash_nonneg', 'pca',"canonical"]
input: output_from("extract_effects")["extracted_effect"]
output: MASH_model = f"{wd}/sumstat/{Theme_prefix}.{effect_model}.V_{vhat}.mash_model.rds",
        resid_corr = f"{wd}/sumstat/{Theme_prefix}.{effect_model}.V_{vhat}.rds",
        flash_output = [f"{wd}/sumstat/{Theme_prefix}.{m}.rds" for m in mixture_components]
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/MASH/mashr_flashr_workflow.ipynb mash \
            --cwd $[wd]/sumstat/ \
            --container $[container] \
            --effect_model $[effect_model] \
            --vhat $[vhat] \
            --output_prefix $[Theme_prefix] \
            --data $[_input] \
            -J $[J] -q $[queue] -c $[yml]

## MASH Posterior

In [ ]:
[MASH_posterior]
parameter: effect_model = 'EZ'
parameter: mixture_components = ['flash', 'flash_nonneg', 'pca',"canonical"]
analysis_units = f'{wd}/sumstat/merged_analysis_unit.txt'
regions = [x.replace("\"","").strip().split() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]
gene = [x[0] for x in regions]
input: output_from("MASH_FLASH")["MASH_model"], regions
output: mash_output_list = f'{wd}/sumstat/mash_output_list'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/MASH/mash_posterior.ipynb  posterior \
            --cwd $[wd]/sumstat/ \
            --container $[container] \
            --mash_model $[_input[0]] \
            --vhat $[vhat] \
            --output_prefix $[Theme_prefix] \
            --posterior_input  $[" ".join(gene)] \
            -J $[J] -q $[queue] -c $[yml]

## Mixture Prior

In [ ]:
[Mixture_prior]
parameter: effect_model = 'EZ'
parameter: mixture_components = ['flash', 'flash_nonneg', 'pca',"canonical"]
parameter: mixture_prior_method = "ed_bovy"
input: output_from("MASH_FLASH")["MASH_model"], output_from("extract_effects")["extracted_effect"]
analysis_units = f'{wd}/sumstat/merged_analysis_unit.txt'
regions = [x.replace("\"","").strip().split() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]
output: mixture_prior = f'{wd}/../Fine_Mapping/Mixture_Prior/{Theme_prefix}.{mixture_prior_method}.V_{vhat}.rds'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/MASH/mixture_prior.ipynb $[mixture_prior_method] \
            --cwd $[wd] \
            --container $[container] \
            --name $[Theme_prefix] \
            --vhat $[vhat] \
            --data  $[_input[1]] \
            --name $[Theme_prefix] \
            -J $[J] -q $[queue] -c $[yml]

## RDS to VCF

In [ ]:
[RDS2VCF]
parameter: data_dir = path("/")
input: output_from("MASH_posterior")["mash_output_list"]
output: f'{wd}/mash_vcf/vcf_output_list.txt'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
        sos run $[exe_dir]/Meta_Analysis/Reformatting/RDS_to_vcf.ipynb rds_to_vcf \
            --wd $[wd]/ \
            --container $[container] \
            --name $[Theme_prefix] \
            --analysis_units $[_input] \
            --data_dir $[data_dir]  \
            -J $[J] -q $[queue] -c $[yml]